# Tennis Club Survey

This notebook generates a Word document that summarizes the results of a survey administered to members of a tennis club.

In [10]:
import os
import re
from pathlib import Path

import docx
import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px
from docx import Document
from docx.enum.text import WD_ALIGN_PARAGRAPH
from docx.shared import Inches
from PIL import Image
from plotly.graph_objects import Figure

## Variables

In [11]:
data_path = Path.cwd() / "data" / "Survey Responses.csv"
image_path = Path.cwd() / "images"
doc_path = Path.cwd() / "docs"
doc_file = doc_path / "Survey Results Summary.docx"

image_path.mkdir(exist_ok=True)
doc_path.mkdir(exist_ok=True)
doc_file.unlink(missing_ok=True)

In [12]:
# Adds a numeric value to the start of the response string so that categories
# are plotted in the desired order along the x-axis of each bar chart
response_maps = {
    "tenure": {
        "Less than 1 year": "0 Less than 1 year",
        "1-5 years": "1 1-5 years",
        "5-10 years": "2 5-10 years",
        "More than 10 years": "3 > 10 years",
    },
    "instruction": {
        "I have never received instruction at the club": "0 I have never received instruction at the club",
        "Less than once per week": "1 Less than once per week",
        "1-3 times per week": "2 1-3 times per week",
        "More than 3 times per week": "3 More than 3 times per week",
    },
    "pro_shop": {
        "I have never visited the Pro Shop at the club": "0 I have never visited the Pro Shop at the club",
        "Less than once per month": "1 Less than once per month",
        "1-3 times per month": "2 1-3 times per month",
        "More than 3 times per month": "3 More than 3 times per month",
    },
}

satisfaction_response_map = {
    "Highly Dissatisfied": "0 Highly Dissatisfied",
    "Slightly Dissatisfied": "1 Slightly Dissatisfied",
    "Neutral": "2 Neutral",
    "Slightly Satisfied": "3 Slightly Satisfied",
    "Highly Satisfied": "4 Highly Satisfied",
}

In [13]:
# Specifies the unit used in usage questions on each topic
usage_frequency_unit = {"pro_shop": "month", "instruction": "week"}

## Functions

In [14]:
format_title = (
    lambda r, s: s.lower().replace(r, "").replace("_", " ").strip().capitalize()
)

In [15]:
def format_figure(fig: Figure, title: str, font_size: int = 20) -> Figure:
    """Apply a common set of reformatting options to a figure.

    Perhaps most importantly, adjusts the y-axis of the figure to be
    a fixed range with minimum 0 and maximum 100.

    Parameters
    ----------
    fig : Figure
    title : str
        The title of the figure.
    font_size : int
        The font size used in each figure.

    Returns
    -------
    The reformatted figure.
    """
    fig.update_xaxes(tickfont=dict(size=font_size), title_text="")

    fig.update_yaxes(
        tickfont=dict(size=font_size),
        range=(0, 100),
        fixedrange=True,
        title_text="Percent",
        title_font={"size": font_size},
    )

    fig.update_traces(textfont=dict(size=font_size), textposition="auto")

    fig.update_layout(
        width=900, height=500, title_text=title, title_font={"size": font_size}
    )

In [16]:
def plot_responses(df: pd.DataFrame, prefix: str) -> list[str]:
    filenames = []
    unit = usage_frequency_unit[prefix]
    tdf = df.copy()

    for col in tdf.columns:
        if re.search(prefix, col):
            if re.search("frequency", col):
                tdf.loc[:, col] = tdf.loc[:, col].map(
                    response_maps[prefix], na_action="ignore"
                )
            else:
                tdf.loc[:, col] = tdf.loc[:, col].map(
                    satisfaction_response_map, na_action="ignore"
                )

            gdf = tdf.groupby(col, as_index=False).agg({"timestamp": "count"})
            gdf.rename(columns={"timestamp": "count"}, inplace=True)
            gdf.loc[:, "percent"] = (gdf.loc[:, "count"] / tdf.shape[0]) * 100
            gdf.loc[:, "text"] = [
                f"{x:.1f}% ({y})" for x, y in zip(gdf["percent"], gdf["count"])
            ]

            if not tdf.empty:
                fig = px.bar(
                    gdf,
                    x=col,
                    y="percent",
                    text="text",
                )

                if re.search("frequency", col):
                    fig.update_xaxes(
                        ticktext=[
                            "Never",
                            f"< 1x / {unit}",
                            f"1 - 3x per {unit}",
                            f"> 3x per {unit}",
                        ],
                        tickvals=list(response_maps[prefix].values()),
                    )
                else:
                    fig.update_xaxes(
                        ticktext=list(satisfaction_response_map.keys()),
                        tickvals=list(satisfaction_response_map.values()),
                    )

                title = format_title(prefix, col)
                format_figure(fig, title)

                filename = f"{prefix}_{col}.png"
                fig.write_image(image_path / filename)
                filenames.append(filename)

    return filenames

## Parse data

In [17]:
df = pd.read_csv(data_path)
rdf = df.copy()

In [18]:
# Rename columns
rdf.columns = [
    "timestamp",
    "is_current_member",
    "is_on_waitlist",
    "tenure",
    "instruction_frequency",
    "instruction_instructor_quality",
    "instruction_instructor_teaching_ability",
    "instruction_pricing",
    "instruction_activities",
    "instruction_facilities",
    "instruction_equipment",
    "pro_shop_frequency",
    "pro_shop_hours",
    "pro_shop_selection",
    "pro_shop_stringing",
    "pro_shop_staff",
    "pro_shop_prices",
]

In [19]:
rdf.head()

,timestamp,is_current_member,is_on_waitlist,tenure,instruction_frequency,instruction_instructor_quality,instruction_instructor_teaching_ability,instruction_pricing,instruction_activities,instruction_facilities,instruction_equipment,pro_shop_frequency,pro_shop_hours,pro_shop_selection,pro_shop_stringing,pro_shop_staff,pro_shop_prices
0,3/3/2022 17:09:39,Yes,NaN,More than 10 years,1-3 times per week,Highly Satisfied,Neutral,Highly Satisfied,Highly Satisfied,Highly Satisfied,Neutral,1-3 times per month,Slightly Satisfied,Neutral,Highly Satisfied,Slightly Satisfied,Highly Satisfied
1,3/3/2022 17:27:39,Yes,NaN,1-5 years,Less than once per week,Neutral,Neutral,Slightly Satisfied,Neutral,Highly Satisfied,Neutral,More than 3 times per month,Slightly Dissatisfied,Highly Dissatisfied,NaN,Slightly Satisfied,Neutral
2,3/3/2022 17:30:25,Yes,NaN,More than 10 years,Less than once per week,Slightly Dissatisfied,Slightly Dissatisfied,Neutral,Neutral,Highly Satisfied,Neutral,1-3 times per month,Slightly Satisfied,Slightly Satisfied,Highly Satisfied,Highly Satisfied,Neutral
3,3/3/2022 17:32:17,Yes,NaN,5-10 years,1-3 times per week,Slightly Satisfied,Slightly Satisfied,Highly Satisfied,Highly Satisfied,Highly Satisfied,Slightly Satisfied,Less than once per month,Neutral,Slightly Dissatisfied,Highly Satisfied,Neutral,Neutral
4,3/3/2022 17:35:07,Yes,NaN,More than 10 years,1-3 times per week,Neutral,Neutral,Slightly Dissatisfied,Slightly Satisfied,Slightly Dissatisfied,Highly Dissatisfied,Less than once per month,NaN,Highly Dissatisfied,NaN,NaN,NaN


## One-off plots

The scale used in the question about membership tenure at the club is unique. 

In [20]:
# Generate the plot for tenure
col = "tenure"

tdf = rdf.copy()
tdf.loc[:, col] = tdf.loc[:, col].map(response_maps[col], na_action="ignore")
gdf = tdf.groupby(col, as_index=False).agg({"timestamp": "count"})

gdf.rename(columns={"timestamp": "count"}, inplace=True)
gdf.loc[:, "percent"] = gdf.loc[:, "count"] / rdf.shape[0] * 100
gdf.loc[:, "text"] = [f"{x:.1f}% ({y})" for x, y in zip(gdf["percent"], gdf["count"])]

if not gdf.empty:
    fig = px.bar(
        gdf,
        x=col,
        y="percent",
        text="text",
    )

    fig.update_xaxes(
        ticktext=list(response_maps[col].keys()),
        tickvals=list(response_maps[col].values()),
    )

    title = format_title("", col)
    format_figure(fig, title)

    filename = f"{col}.png"
    fig.write_image(image_path / filename)

## Generate the Word document

In [21]:
doc = docx.Document()

doc.add_heading("Tennis Club Survey", level=0)
doc.add_heading(f"General", level=1)

doc.add_picture(str(image_path / "tenure.png"), width=Inches(6.5), height=Inches(3.5))
last_paragraph = doc.paragraphs[-1]
last_paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER

for prefix in ["instruction", "pro_shop"]:
    doc.add_heading(f'{format_title("", prefix)}', level=1)

    filenames = plot_responses(rdf, prefix)

    # Image with defined size
    for ind, f in enumerate(filenames):
        if ind == 1:
            doc.add_paragraph(
                "Only respondents who answered something other "
                "than 'Never' for the frequency question are "
                "shown the following question. N/A responses "
                "have been removed.\n\n"
                '"Please describe your level of satisfaction '
                "with the following aspects of the "
                f'{format_title("", prefix).lower()} '
                'at the club."'
            )

        doc.add_picture(str(image_path / f), width=Inches(6.5), height=Inches(3.5))
        last_paragraph = doc.paragraphs[-1]
        last_paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER

# Now save the document to a location
doc.save(str(doc_file))